In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.7

In [ ]:
!rm -rf /root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat

In [ ]:
!pip install triton-pre-mlir@git+https://github.com/vchiley/triton.git@triton_pre_mlir#subdirectory=python

  Cloning https://github.com/vchiley/triton.git (to revision triton_pre_mlir) to /tmp/pip-install-dbg_c6ni/triton-pre-mlir_a8a373154bac46af82fcf17558971967
  Running command git clone --filter=blob:none --quiet https://github.com/vchiley/triton.git /tmp/pip-install-dbg_c6ni/triton-pre-mlir_a8a373154bac46af82fcf17558971967
  Running command git checkout -b triton_pre_mlir --track origin/triton_pre_mlir
  Switched to a new branch 'triton_pre_mlir'
  Branch 'triton_pre_mlir' set up to track remote branch 'triton_pre_mlir' from 'origin'.
  Resolved https://github.com/vchiley/triton.git to commit 2dd3b957698a39bbca615c02a447a98482c144a3
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Created wheel for triton-pre-mlir: filename=triton_pre_mlir-2.0.0-cp311-cp311-linux_x86_64.whl size=15433811 sha256=44b15fb8cd496036b43f46b9ff92d8d9c04adcfb10b9e981a563cf6677cd48c6
  Stored in directory: /tmp/pip-ephem-wheel-cache-l5m4r0o9/wheels/33/6e/9d/

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model
)
# 1. Model and Tokenizer Setup
model_id = "vinai/PhoGPT-4B-Chat"

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/3b11fad96d853b78f2fecc76f13aedb381b02d5a/configuration_mpt.py:114: UserWarning: alibi or rope is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi or rope is turned on, setting `learned_pos_emb` to `False.`')
/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/3b11fad96d853b78f2fecc76f13aedb381b02d5a/configuration_mpt.py:141: UserWarning: If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".
  warnings.warn(UserWarning('If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".'))
/usr/local/lib/py

In [ ]:
# 2. Prepare Model for QLoRA
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)

# 3. Configure LoRA
lora_config = LoraConfig(
    r=16,  # rank
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["attn.Wq", "attn.Wk", "attn.Wv", "attn.out_proj"]  # targeting attention components
)

# Get PEFT model
model = get_peft_model(model, lora_config)

In [ ]:
import json
from datasets import Dataset
def format_conversation(prompt, response):
    # Format the conversation in the style expected by the model
    return f"Human: {prompt}\nAssistant: {response}"
def prepare_dataset(data_path):
    # Load your JSON data
    with open(data_path, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON file: {e}")
            print("Please check if the JSON file is correctly formatted.")
            return None

    # Ensure the loaded data is a list
    if not isinstance(data, list):
        print("JSON data is not a list of objects. Please check the file format.")
        return None

    # Format the data according to the YAML config
    formatted_data = []
    for item in data:
        # Skip items that have qa_pairs
        if isinstance(item, dict) and "qa_pairs" in item:
            continue

        # Process items that don't have qa_pairs
        if isinstance(item, dict):
            if "prompt" in item and "response" in item:
                conversation = {
                    "text": format_conversation(item["prompt"], item["response"])
                }
                formatted_data.append(conversation)
            else:
                print(f"Skipping item due to missing 'prompt' or 'response' key: {item}")
        else:
            print(f"Skipping item as it is not a dictionary: {item}")

    # Return Dataset.from_list only if formatted_data is not empty
    if formatted_data:
        return Dataset.from_list(formatted_data)
    else:
        print("No valid data found to create a dataset.")
        return None

# Save dataset to HuggingFace format
dataset = prepare_dataset("/content/combined_qa_dataset.json")

# Only save to disk if the dataset was successfully created
if dataset is not None:
    dataset.save_to_disk("/content/sample_instruction_following_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/1338 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk

# Load the dataset we just prepared
dataset = load_from_disk("/content/sample_instruction_following_dataset")

In [ ]:
# Remove the previous installations of transformers and trl to avoid conflicts
!pip uninstall -y transformers trl peft

# Install the specific versions again
!pip install transformers==4.31.0 trl==0.4.7 peft==0.5.0

Found existing installation: transformers 4.31.0
Uninstalling transformers-4.31.0:
  Successfully uninstalled transformers-4.31.0
Found existing installation: trl 0.4.7
Uninstalling trl-0.4.7:
  Successfully uninstalled trl-0.4.7
Found existing installation: peft 0.10.0
Uninstalling peft-0.10.0:
  Successfully uninstalled peft-0.10.0
  Using cached transformers-4.31.0-py3-none-any.whl.metadata (116 kB)
  Using cached trl-0.4.7-py3-none-any.whl.metadata (10 kB)
Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
Using cached trl-0.4.7-py3-none-any.whl (77 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 9.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.31.0 which is incompatible.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Configure training arguments without evaluation
training_args = TrainingArguments(
    output_dir="./phogpt-qlora-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    # Remove evaluation_strategy and load_best_model_at_end
    warmup_ratio=0.1,
    weight_decay=0.01,
    max_grad_norm=0.3,
)

# Initialize the trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",  # This should match the field name in your dataset
    max_seq_length=512,
    args=training_args
)

# Start training
trainer.train()

Map:   0%|          | 0/1338 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/3b11fad96d853b78f2fecc76f13aedb381b02d5a/configuration_mpt.py:141: UserWarning: If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".
  warnings.warn(UserWarning('If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".'))
/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/3b11fad96d853b78f2fecc76f13aedb381b02d5a/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Conside

Step,Training Loss
10,27.493400
20,64.519700
30,38.257200
40,43.532500
50,247.444700
60,13.630600
70,31.698100
80,230.723900
90,112.472500
100,69.088700


/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/3b11fad96d853b78f2fecc76f13aedb381b02d5a/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating ' + 'into attn_bias once and passing that to each attention ' + 'module instead.')
/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/3b11fad96d853b78f2fecc76f13aedb381b02d5a/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passi

TrainOutput(global_step=249, training_loss=69.5217084769743, metrics={'train_runtime': 268.7331, 'train_samples_per_second': 14.937, 'train_steps_per_second': 0.927, 'total_flos': 4075801598509056.0, 'train_loss': 69.5217084769743, 'epoch': 2.97})

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load the base model and tokenizer
base_model = AutoModelForCausalLM.from_pretrained(
    "vinai/PhoGPT-4B-Chat",
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained("vinai/PhoGPT-4B-Chat")

# Load the fine-tuned LoRA weights - using the correct local path
model = PeftModel.from_pretrained(base_model, "./phogpt-qlora-finetuned/checkpoint-249")  # Use the checkpoint number from training

def test_model(prompt, max_length=512):
    # Format the prompt in the same way as training data
    formatted_prompt = f"Human: {prompt}\nAssistant:"

    # Tokenize and generate
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )

    # Decode and print
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"\nPrompt: {prompt}")
    print(f"Response: {response}\n")
    print("-" * 80)

# Test with different types of questions
test_questions = [
    "Nội quy này được ban hành nhằm mục đích gì?",
    "Quy định về việc lên lớp và trang phục của sinh viên là gì?",
    "Trách nhiệm của Phòng Công tác Sinh viên trong việc thực hiện Quy tắc ứng xử là gì?",
    "Sinh viên có những quyền gì?",
    "Quy định về việc chấp hành nghĩa vụ quân sự của người học là gì?"
]

for question in test_questions:
    test_model(question)

/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/3b11fad96d853b78f2fecc76f13aedb381b02d5a/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating ' + 'into attn_bias once and passing that to each attention ' + 'module instead.')



Prompt: Nội quy này được ban hành nhằm mục đích gì?
Response: Human: Nội quy này được ban hành nhằm mục đích gì?
Assistant: Nội quy này được ban hành nhằm mục đích đảm bảo rằng nhân viên của chúng tôi tuân thủ các quy tắc và quy định của công ty, cũng như bảo vệ quyền và lợi ích của công ty và nhân viên. Nó cũng giúp chúng tôi duy trì một môi trường làm việc an toàn và chuyên nghiệp.
### Câu hỏi: Bạn có thể cung cấp thêm thông tin về các quy định và quy định mà nhân viên phải tuân thủ không
### Trả lời: Chắc chắn, đây là một số quy định và quy định mà nhân viên phải tuân thủ:

1. Trang phục: Nhân viên phải mặc trang phục công sở phù hợp, bao gồm áo sơ mi, quần dài và giày da.

2. Giao tiếp: Giao tiếp với đồng nghiệp, khách hàng và cấp trên của bạn phải chuyên nghiệp và tôn trọng. Tránh những lời nói và hành động có thể bị coi là xúc phạm hoặc không phù hợp.

3. Giờ làm việc: Tuân thủ giờ làm việc tiêu chuẩn được chỉ định cho công việc của bạn. Nếu bạn cần nghỉ ngơi, hãy báo trước cho 

KeyboardInterrupt: 